In [1]:
import json

In [3]:
with open('data/mw24_100p_dev.json') as f:
    data = json.load(f)
len(data)

7374

In [19]:
with open('outputs/runs/smapling_exp/train/5p/example_4_normal/running_log.json') as f:
    data.extend(json.load(f))

In [10]:
data.extend(tmp)

In [20]:
len(data)

2731

In [21]:
with open('data/optimal_train_5p_v1_normal.json', 'w') as f:
    json.dump(data, f)

In [25]:
with open('data/mw21_10p_train_v1.1.json', 'w') as f:
    json.dump(data[:1374], f)
with open('data/mw21_10p_train_v1.2.json', 'w') as f:
    json.dump(data[1374: 2748], f)
with open('data/mw21_10p_train_v1.3.json', 'w') as f:
    json.dump(data[2748:4120], f)
with open('data/mw21_10p_train_v1.4.json', 'w') as f:
    json.dump(data[4120:], f)

# data[:1374]
# data[1374: 2748]
# data[2748:4120]
# data[4120:]

In [19]:
with open('data/mw21_1p_train_v1.1.json', 'w') as f:
    json.dump(data[:134], f)
with open('data/mw21_1p_train_v1.2.json', 'w') as f:
    json.dump(data[134:263], f)
with open('data/mw21_1p_train_v1.3.json', 'w') as f:
    json.dump(data[263:391], f)
with open('data/mw21_1p_train_v1.4.json', 'w') as f:
    json.dump(data[391:], f)
# data[:134]
# data[134:263]
# data[263:391]
# data[391: ]

In [37]:
with open('data/mw21_5p_train_v1.1.json', 'w') as f:
    json.dump(data[:684], f)

with open('data/mw21_5p_train_v1.2.json', 'w') as f:
    json.dump(data[684:1370], f)

with open('data/mw21_5p_train_v1.3.json', 'w') as f:
    json.dump(data[1370:2059], f)

with open('data/mw21_5p_train_v1.4.json', 'w') as f:
    json.dump(data[2059:], f)
    

# d_1 = data[:684]
# d_2 = data[684:1370]
# d_3 = data[1370:2059]
# d_4 = data[2059:]

In [33]:
d_1[0]['turn_id'], d_1[-1]['turn_id']

(0, 6)

In [34]:
d_2[0]['turn_id'], d_2[-1]['turn_id']

(0, 5)

In [35]:
d_3[0]['turn_id'], d_3[-1]['turn_id']

(0, 9)

In [36]:
d_4[0]['turn_id'], d_4[-1]['turn_id']

(0, 7)

In [1]:
from vllm import LLM, SamplingParams, EngineArgs, LLMEngine
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM
import torch
from vllm.usage.usage_lib import UsageContext
from typing import ClassVar, List, Optional, Sequence, Union, cast, overload

from vllm.engine.arg_utils import EngineArgs
from vllm.engine.llm_engine import LLMEngine
# from vllm.inputs import PromptStrictInputs
from vllm.sampling_params import SamplingParams
from vllm.usage.usage_lib import UsageContext
from vllm.model_executor.layers.quantization.gptq import GPTQConfig
from transformers import BitsAndBytesConfig
from datetime import datetime

import os, json

# # os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# # from vllm.model_executor.layers.quantization.bitsandbytes import BitsAndBytesConfig

In [2]:
import openai

In [3]:
openai.api_key ="sk-proj-YQt8oIaT6sRs0AkUbctNT3BlbkFJoCQS25v6e5lLnNKDZpZL"

In [1]:
prompt_text = [
        {
          "role": "system",
          "content": "**Task:** You are an expert in Dialogue State Tracking (DST) focused on managing and updating the dialogue state change based on system-user interactions. The dialogue state represents the user's preferences and booking details across different domains: Hotel, Train, Attraction, Restaurant, and Taxi.\n\n"
        },
        {
          "role": "user",
          "content": "**Guidelines:**\n\n    1. **Hotel:**\n        - **Slots:** name (string), pricerange (PriceRange), type (HotelType), parking (Option), book stay (integer), book day (DayOfWeek), book people (integer), area (Area), stars (integer between 0 and 5 or \"dontcare\"), internet (Option).\n        - **Valid Values:**\n            - PriceRange: \"dontcare\", \"cheap\", \"moderate\", \"expensive\".\n            - HotelType: \"hotel\", \"guest house\", \"dontcare\".\n            - Option: \"yes\", \"no\", \"dontcare\".\n            - DayOfWeek: \"monday\", \"tuesday\", \"wednesday\", \"thursday\", \"friday\", \"saturday\", \"sunday\".\n            - Area: \"dontcare\", \"centre\", \"east\", \"north\", \"south\", \"west\".\n\n    2. **Train:**\n        - **Slots:** destination (string), departure (string), day (DayOfWeek), book people (integer), leaveat (hh:mm or \"dontcare\"), arriveby (hh:mm or \"dontcare\").\n\n    3. **Attraction:**\n        - **Slots:** name (string), area (Area), type (AttractionType).\n        - **Valid Values:**\n            - AttractionType: \"architecture\", \"boat\", \"church\", \"cinema\", \"college\", \"concert hall\", \"entertainment\", \"hotspot\", \"multiple sports\", \"museum\", \"nightclub\", \"park\", \"special\", \"swimming pool\", \"theatre\", \"dontcare\".\n\n    4. **Restaurant:**\n        - **Slots:** name (string), food (string), pricerange (PriceRange), area (Area), book time (hh:mm or \"dontcare\"), book day (DayOfWeek), book people (integer).\n\n    5. **Taxi:**\n        - **Slots:** destination (string), departure (string), leaveat (hh:mm or \"dontcare\"), arriveby (hh:mm or \"dontcare\").\n\n\n**Example 1 of Dialogue State Change Update Task:**\n    **Previous Belief State (Before the Latest User Interaction):** \n        {}\n\n    **Latest Conversation Between System and User:** \n        **User:** \"are there many place -s to stay with mid range price -s ?\"\n\n    **Instructions:**\n        - Based on the user\u2019s latest input, update the belief state by correctly identifying and filling in the relevant domain(s), slot(s) and value(s).\n        - Provide your output strictly in the Required Output Format below.\n\n    **Required Output Format:**\n        **Dialogue state change after Latest Conversation Between System and User:** \n"
        },
        {
          "role": "assistant",
          "content": "            {'hotel-pricerange': 'moderate'}\n"
        },
        {
          "role": "user",
          "content": "\n**Example 2 of Dialogue State Change Update Task:**\n    **Previous Belief State (Before the Latest User Interaction):** \n        {'taxi-destination': 'kohinoor', 'taxi-departure': 'university arms hotel', 'taxi-arriveby': '12:00', 'restaurant-book day': 'thursday', 'restaurant-book people': '5', 'restaurant-book time': '12:00', 'restaurant-food': 'indian', 'restaurant-pricerange': 'cheap', 'restaurant-name': 'kohinoor', 'restaurant-area': 'centre', 'hotel-name': 'university arms hotel', 'hotel-parking': 'yes', 'hotel-stars': '4', 'hotel-type': 'hotel'}\n\n    **Latest Conversation Between System and User:** \n        **System:** \"ok your all set ! a yellow bmw will be picking you up a few minutes before your reservation at the restaurant . contact number is 07785034458 . anything else ?\"\n        **User:** \"no that will be all thank you\"\n\n    **Instructions:**\n        - Based on the user\u2019s latest input, update the belief state by correctly identifying and filling in the relevant domain(s), slot(s) and value(s).\n        - Provide your output strictly in the Required Output Format below.\n\n    **Required Output Format:**\n        **Dialogue state change after Latest Conversation Between System and User:** \n"
        },
        {
          "role": "assistant",
          "content": "            {}\n"
        },
        {
          "role": "user",
          "content": "\n**Example 3 of Dialogue State Change Update Task:**\n    **Previous Belief State (Before the Latest User Interaction):** \n        {'hotel-name': 'acorn guest house', 'hotel-area': 'north', 'hotel-parking': 'no', 'hotel-stars': '4', 'hotel-type': 'guest house', 'attraction-type': 'museum', 'attraction-name': 'cambridge artworks'}\n\n    **Latest Conversation Between System and User:** \n        **System:** \"the postcode is cb41da . is there anything else i can help you with today ?\"\n        **User:** \"yeah , can you have a taxi pick me up at the acorn and take me to the artworks , say around 24:45 ?\"\n\n    **Instructions:**\n        - Based on the user\u2019s latest input, update the belief state by correctly identifying and filling in the relevant domain(s), slot(s) and value(s).\n        - Provide your output strictly in the Required Output Format below.\n\n    **Required Output Format:**\n        **Dialogue state change after Latest Conversation Between System and User:** \n"
        },
        {
          "role": "assistant",
          "content": "            {'taxi-leaveat': '24:45', 'taxi-destination': 'cambridge artworks', 'taxi-departure': 'acorn guest house'}\n"
        },
        {
          "role": "user",
          "content": "\n**Example 4 of Dialogue State Change Update Task:**\n    **Previous Belief State (Before the Latest User Interaction):** \n        {'taxi-leaveat': '15:45', 'taxi-destination': 'acorn guest house', 'taxi-departure': 'sesame restaurant and bar', 'restaurant-name': 'sesame restaurant and bar', 'hotel-name': 'acorn guest house', 'hotel-area': 'north', 'hotel-stars': '4', 'hotel-type': 'guest house'}\n\n    **Latest Conversation Between System and User:** \n        **System:** \"do you need any further assistance today ?\"\n        **User:** \"no , that is all . thank you .\"\n\n    **Instructions:**\n        - Based on the user\u2019s latest input, update the belief state by correctly identifying and filling in the relevant domain(s), slot(s) and value(s).\n        - Provide your output strictly in the Required Output Format below.\n\n    **Required Output Format:**\n        **Dialogue state change after Latest Conversation Between System and User:** \n"
        },
        {
          "role": "assistant",
          "content": "            {}\n"
        },
        {
          "role": "user",
          "content": "\n**Example 5 of Dialogue State Change Update Task:**\n    **Previous Belief State (Before the Latest User Interaction):** \n        {'train-book people': '6', 'train-leaveat': '18:30', 'train-destination': 'cambridge', 'train-day': 'tuesday', 'train-departure': 'leicester'}\n\n    **Latest Conversation Between System and User:** \n        **System:** \"okay , the booking was successful . your reference number is m3uvolvxd . the total fee of 226.8 gbp is payable at the station .\"\n        **User:** \"thanks ! i am also looking for a place to stay that s a guest house .\"\n\n    **Instructions:**\n        - Based on the user\u2019s latest input, update the belief state by correctly identifying and filling in the relevant domain(s), slot(s) and value(s).\n        - Provide your output strictly in the Required Output Format below.\n\n    **Required Output Format:**\n        **Dialogue state change after Latest Conversation Between System and User:** \n"
        },
        {
          "role": "assistant",
          "content": "            {'hotel-type': 'guest house'}\n"
        },
        {
          "role": "user",
          "content": "\n**Example 6 of Dialogue State Change Update Task:**\n    **Previous Belief State (Before the Latest User Interaction):** \n        {'restaurant-book day': 'monday', 'restaurant-book people': '2', 'restaurant-book time': '13:15', 'restaurant-pricerange': 'cheap', 'restaurant-name': 'the lucky star', 'restaurant-area': 'south', 'attraction-type': 'museum', 'attraction-name': 'byard art'}\n\n    **Latest Conversation Between System and User:** \n        **System:** \"i can help you with that . i'll need to know your departure site and destination . you can also tell me the departure time and arrival time .\"\n        **User:** \"i would like to get the postcode for the museum first , please . i'll need the taxi to get from the museum to lucky star in time for my booking .\"\n\n    **Instructions:**\n        - Based on the user\u2019s latest input, update the belief state by correctly identifying and filling in the relevant domain(s), slot(s) and value(s).\n        - Provide your output strictly in the Required Output Format below.\n\n    **Required Output Format:**\n        **Dialogue state change after Latest Conversation Between System and User:** \n"
        },
        {
          "role": "assistant",
          "content": "            {'taxi-destination': 'the lucky star', 'taxi-departure': 'byard art', 'taxi-arriveby': '13:15'}\n"
        },
        {
          "role": "user",
          "content": "\n**Example 7 of Dialogue State Change Update Task:**\n    **Previous Belief State (Before the Latest User Interaction):** \n        {'hotel-internet': 'no', 'hotel-type': 'guest house', 'attraction-type': 'architecture'}\n\n    **Latest Conversation Between System and User:** \n        **System:** \"the alpha milton guest house is a guest house in the north area and is moderate -ly priced . it does not have parking or internet . would you like to book a room ?\"\n        **User:** \"no . i need free parking and a 4 star rating . would you possibly have a guest house with that .\"\n\n    **Instructions:**\n        - Based on the user\u2019s latest input, update the belief state by correctly identifying and filling in the relevant domain(s), slot(s) and value(s).\n        - Provide your output strictly in the Required Output Format below.\n\n    **Required Output Format:**\n        **Dialogue state change after Latest Conversation Between System and User:** \n"
        },
        {
          "role": "assistant",
          "content": "            {'hotel-parking': 'yes', 'hotel-stars': '4', 'hotel-internet': '[DELETE]'}\n"
        },
        {
          "role": "user",
          "content": "\n**Example 8 of Dialogue State Change Update Task:**\n    **Previous Belief State (Before the Latest User Interaction):** \n        {'restaurant-book day': 'thursday', 'restaurant-book people': '7', 'restaurant-book time': '16:15', 'restaurant-food': 'spanish', 'restaurant-pricerange': 'cheap', 'restaurant-area': 'centre'}\n\n    **Latest Conversation Between System and User:** \n        **System:** \"we do not have another spanish restaurant . would you like chinese or italian perhaps ?\"\n        **User:** \"can you look for chinese restaurant -s please .\"\n\n    **Instructions:**\n        - Based on the user\u2019s latest input, update the belief state by correctly identifying and filling in the relevant domain(s), slot(s) and value(s).\n        - Provide your output strictly in the Required Output Format below.\n\n    **Required Output Format:**\n        **Dialogue state change after Latest Conversation Between System and User:** \n"
        },
        {
          "role": "assistant",
          "content": "            {'restaurant-food': 'chinese'}\n"
        },
        {
          "role": "user",
          "content": "\n**Example 9 of Dialogue State Change Update Task:**\n    **Previous Belief State (Before the Latest User Interaction):** \n        {}\n\n    **Latest Conversation Between System and User:** \n        **User:** \"i want a hotel that has free parking .\"\n\n    **Instructions:**\n        - Based on the user\u2019s latest input, update the belief state by correctly identifying and filling in the relevant domain(s), slot(s) and value(s).\n        - Provide your output strictly in the Required Output Format below.\n\n    **Required Output Format:**\n        **Dialogue state change after Latest Conversation Between System and User:** \n"
        },
        {
          "role": "assistant",
          "content": "            {'hotel-parking': 'yes', 'hotel-type': 'hotel'}\n"
        },
        {
          "role": "user",
          "content": "\n**Example 10 of Dialogue State Change Update Task:**\n    **Previous Belief State (Before the Latest User Interaction):** \n        {'train-destination': 'london kings cross', 'train-day': 'wednesday'}\n\n    **Latest Conversation Between System and User:** \n        **System:** \"i can help with that . where are you departing from ?\"\n        **User:** \"i will be departing from cambridge , and need to arrive by 08:30\"\n\n    **Instructions:**\n        - Based on the user\u2019s latest input, update the belief state by correctly identifying and filling in the relevant domain(s), slot(s) and value(s).\n        - Provide your output strictly in the Required Output Format below.\n\n    **Required Output Format:**\n        **Dialogue state change after Latest Conversation Between System and User:** \n"
        },
        {
          "role": "assistant",
          "content": "            {'train-arriveby': '08:30', 'train-departure': 'cambridge'}\n"
        },
        {
          "role": "user",
          "content": "\n**Example 11 of Dialogue State Change Update Task:**\n    **Previous Belief State (Before the Latest User Interaction):** \n        {'restaurant-food': 'italian', 'restaurant-area': 'centre', 'restaurant-pricerange': 'dontcare', 'restaurant-book people': '4', 'restaurant-book time': '18:30', 'restaurant-book day': 'saturday', 'attraction-type': 'dontcare', 'attraction-area': 'centre'}\n\n    **Latest Conversation Between System and User:** \n        **System:** \"how about corpus christi college ? its address is saint andrew s street , postcode is cb21rh , and their phone number is 01223338000 .\"\n        **User:** \"what about the entrance fee ? is there 1 ?\"\n\n    **Instructions:**\n        - Based on the user\u2019s latest input, update the belief state by correctly identifying and filling in the relevant domain(s), slot(s) and value(s).\n        - Provide your output strictly in the Required Output Format below.\n\n    **Required Output Format:**\n        **Dialogue state change after Latest Conversation Between System and User:** \n"
        }
      ]

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

In [5]:
prompt_ids = [tokenizer.apply_chat_template(prompt_text, add_generation_prompt=True, return_tensors="pt")]

In [7]:
[tokenizer.batch_decode(prompt, skip_special_tokens=False)[0] for prompt in prompt_ids]

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n**Task:** You are an expert in Dialogue State Tracking (DST) focused on managing and updating the dialogue state change based on system-user interactions. The dialogue state represents the user\'s preferences and booking details across different domains: Hotel, Train, Attraction, Restaurant, and Taxi.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n**Guidelines:**\n\n    1. **Hotel:**\n        - **Slots:** name (string), pricerange (PriceRange), type (HotelType), parking (Option), book stay (integer), book day (DayOfWeek), book people (integer), area (Area), stars (integer between 0 and 5 or "dontcare"), internet (Option).\n        - **Valid Values:**\n            - PriceRange: "dontcare", "cheap", "moderate", "expensive".\n            - HotelType: "hotel", "guest house", "dontcare".\n            - Option: "yes", "no", "dontcare".\n            - DayOfWeek: "monday", "tuesday", "wednesday", "thursday", "friday", "satu

In [11]:
tokenizer.batch_decode(prompt_ids[0], skip_special_tokens=False)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n**Task:** You are an expert in Dialogue State Tracking (DST) focused on managing and updating the dialogue state change based on system-user interactions. The dialogue state represents the user\'s preferences and booking details across different domains: Hotel, Train, Attraction, Restaurant, and Taxi.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n**Guidelines:**\n\n    1. **Hotel:**\n        - **Slots:** name (string), pricerange (PriceRange), type (HotelType), parking (Option), book stay (integer), book day (DayOfWeek), book people (integer), area (Area), stars (integer between 0 and 5 or "dontcare"), internet (Option).\n        - **Valid Values:**\n            - PriceRange: "dontcare", "cheap", "moderate", "expensive".\n            - HotelType: "hotel", "guest house", "dontcare".\n            - Option: "yes", "no", "dontcare".\n            - DayOfWeek: "monday", "tuesday", "wednesday", "thursday", "friday", "satu

In [31]:
prompt= [
    {"role": "user", "content": "Tell me about any thing that can make me feel good."},
]

In [42]:
import copy 
beam_size = 2
valid_beams = [(prompt, 0)]
complete_beams = []

# Beam Search
for _ in range(120):
    step_valid_beams = []
    for message, score in valid_beams:
        
        args = {
            "model": 'gpt-3.5-turbo-0125',
            "messages": message,
            "max_tokens": 1,
            "logprobs": True,
            "temperature": 0.8,
            # "top_logprobs": 20,
            "stop":  ['--', '\n', ';', '#'],
            "n": beam_size
        }
        result = openai.chat.completions.create(**args)

        for i in range(beam_size):
            beam_input = copy.deepcopy(message)
            generated_text = result.choices[i].message.content
            generated_text = generated_text.strip()
            if beam_input[-1]['role'] == 'user':
                beam_input.append({
                    "role": "assistant",
                    "content": generated_text
                })
            else:
                assert beam_input[-1]['role'] == 'assistant'
                beam_input[-1]['content'] += ' ' + generated_text
            
            if result.choices[0].finish_reason == 'stop':
                complete_beams.append((beam_input, score))
            else:
                step_valid_beams.append((beam_input, score + result.choices[i].logprobs.content[-1].logprob[-1]))

    step_valid_beams = sorted(step_valid_beams, key=lambda x: x[1], reverse=True)
    valid_beams = step_valid_beams[:beam_size]

In [39]:
result.choices[0]

Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[], refusal=None), message=ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=None, refusal=None))

In [69]:
prompt= [
    {"role": "user", "content": "Tell me about any thing that can make me feel good."},
    {"role": "assistant", "content": "Sp "}
]

In [76]:
args = {
    "model": 'gpt-3.5-turbo-0125',
    "messages": prompt,
    "max_tokens": 120,
    "logprobs": True,
    "temperature": 0.8,
    # "top_logprobs": 20,
    'n': 2
}
result = openai.chat.completions.create(**args)

In [80]:
result.choices[0].message.content, result.choices[1].message.content

("ending time in nature, whether it's going for a hike, sitting by the beach, or simply taking a walk in a park can make you feel good. The fresh air, natural surroundings, and the sounds of birds chirping can have a calming and uplifting effect on your mood.",
 'ending time with loved ones, practicing gratitude, exercising, spending time in nature, listening to music, helping others, practicing mindfulness and meditation, engaging in hobbies and activities that bring you joy, getting enough sleep, eating nutritious foods, and taking care of your mental and emotional well-being can all help to make you feel good.')

In [68]:
import tiktoken
tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo-0125')
ids = tokenizer.encode(result.choices[0].message.content)

tokenizer.decode([ids[0]])


'Sp'

In [63]:
ids[0]

6540

In [41]:
valid_beams

[([{'role': 'user',
    'content': 'Tell me about any thing that can make me feel good.'},
   {'role': 'assistant',
    'content': 'Thereare manythings that canmakeyoufeelgood,suchasspendingtimewithlovedones,practicingself-care activities,exercising,listeningtomusic,meditating,or engaginginhobbies.Additionally, acts ofkindness,helpingothers,and'}],
  -23.61216214260001),
 ([{'role': 'user',
    'content': 'Tell me about any thing that can make me feel good.'},
   {'role': 'assistant',
    'content': 'Thereare manythings that canmakeyoufeelgood,suchasspendingtimewithlovedones,practicingself-care activities,exercising,listeningtomusic,meditating,or engaginginhobbies.Additionally, acts ofkindness,helpingothers,and'}],
  -23.84897044560001)]

In [26]:
sorted([1, 2, 3, 4], reverse=True)

[4, 3, 2, 1]

In [18]:
import tiktoken

tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo-0125')

In [19]:
tokenizer.special_tokens_set

{'<|endofprompt|>',
 '<|endoftext|>',
 '<|fim_middle|>',
 '<|fim_prefix|>',
 '<|fim_suffix|>'}

In [21]:
args = {
            "model": 'gpt-3.5-turbo-0125',
            "messages": prompt_text,
            "max_tokens": 300,
            "logprobs": True,
            "temperature": 0.0,
            "stop":  ['--', '\n', ';', '#'],
            "n": 1
        }

        # result = openai.completions.create(**args)
result = openai.chat.completions.create(**args)
        # completions = dict(zip(

In [23]:
print(result)

ChatCompletion(id='chatcmpl-A0mRKBSSlDlMFqYvCMdqgk9mbFTA4', choices=[Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='           ', bytes=[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32], logprob=-0.0016619664, top_logprobs=[]), ChatCompletionTokenLogprob(token=" {'", bytes=[32, 123, 39], logprob=-0.15444973, top_logprobs=[]), ChatCompletionTokenLogprob(token='attr', bytes=[97, 116, 116, 114], logprob=-2.188868e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token='action', bytes=[97, 99, 116, 105, 111, 110], logprob=-1.2113979e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token='-name', bytes=[45, 110, 97, 109, 101], logprob=-0.12292929, top_logprobs=[]), ChatCompletionTokenLogprob(token="':", bytes=[39, 58], logprob=-0.00014013138, top_logprobs=[]), ChatCompletionTokenLogprob(token=" '", bytes=[32, 39], logprob=-0.00014168103, top_logprobs=[]), ChatCompletionTokenLogprob(token='cor', bytes=[99, 111, 114], logprob=-7.8748824e-0

In [16]:
beam_size = 2
valid_beams = [(prompt_text, 0)]

# Beam Search
for new_input, score in valid_beams:
    print(new_input[-1]['content'])


**Example 11 of Dialogue State Change Update Task:**
    **Previous Belief State (Before the Latest User Interaction):** 
        {'restaurant-food': 'italian', 'restaurant-area': 'centre', 'restaurant-pricerange': 'dontcare', 'restaurant-book people': '4', 'restaurant-book time': '18:30', 'restaurant-book day': 'saturday', 'attraction-type': 'dontcare', 'attraction-area': 'centre'}

    **Latest Conversation Between System and User:** 
        **System:** "how about corpus christi college ? its address is saint andrew s street , postcode is cb21rh , and their phone number is 01223338000 ."
        **User:** "what about the entrance fee ? is there 1 ?"

    **Instructions:**
        - Based on the user’s latest input, update the belief state by correctly identifying and filling in the relevant domain(s), slot(s) and value(s).
        - Provide your output strictly in the Required Output Format below.

    **Required Output Format:**
        **Dialogue state change after Latest Convers

In [12]:
result.choices[0].message.content

'           '

In [9]:
result

ChatCompletion(id='chatcmpl-A0mCgger4yTzy8AHsES9avfdvannz', choices=[Choice(finish_reason='length', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='           ', bytes=[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32], logprob=-0.0011549857, top_logprobs=[])], refusal=None), message=ChatCompletionMessage(content='           ', role='assistant', function_call=None, tool_calls=None, refusal=None))], created=1724748658, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=1, prompt_tokens=3039, total_tokens=3040))

In [24]:
result.choices[0].finish_reason

'stop'

In [2]:
with open('outputs/runs/preliminary/prompt_format/plain_text_8B_0825_1653/running_log.json') as f:
    dataset = json.load(f)

In [2]:
prompts_string = [
        {
            "role": "system",
            "content": "You are an expert in Dialogue State Tracking(DST) and python coding. And also you are a talented prompt engineer. \n Please generate the shots which are the most "
        },
        {
            "role": "user",
            "content": "import abc\nfrom dataclasses import dataclass\nfrom typing import Literal, Union\n\nPriceRange = Literal[\"dontcare\", \"cheap\", \"moderate\", \"expensive\"]\nHotelType = Literal[\"hotel\", \"guest house\", \"dontcare\"]\nOption = Literal[\"yes\", \"no\", \"dontcare\"]\nDayOfWeek = Literal[\"monday\", \"tuesday\", \"wednesday\", \"thursday\", \"friday\", \"saturday\", \"sunday\"]\nArea = Literal[\"dontcare\", \"centre\", \"east\", \"north\", \"south\", \"west\"]\n\n\n@dataclass\nclass Hotel:\n    name: str = None\n    price_range: PriceRange = None\n    type: HotelType = None\n    parking: Option = None\n    book_number_of_days: int = None\n    book_day: DayOfWeek = None\n    book_people: int = None\n    area: Area = None\n    stars: Union[int, Literal[\"dontcare\"]] = None  # between 0 and 5 or dontcare\n    internet: Option = None\n\n\n@dataclass\nclass Train:\n    destination: str = None\n    leave_from: str = None\n    day: DayOfWeek = None\n    book_people: int = None\n    depart_time: str = None  # hh:mm or dontcare\n    arrive_by_time: str = None  # hh:mm or dontcare\n\n\nAttractionType = Literal[\"architecture\", \"boat\", \"church\", \"cinema\", \"college\", \"concert hall\", \"entertainment\",\n                         \"hotspot\", \"multiple sports\", \"museum\", \"nightclub\", \"park\", \"special\", \"swimming pool\",\n                         \"theatre\", \"dontcare\"]\n\n\n@dataclass\nclass Attraction:\n    name: str = None\n    area: Area = None\n    type: AttractionType = None\n\n\n@dataclass\nclass Restaurant:\n    name: str = None\n    food_type: str = None\n    price_range: PriceRange = None\n    area: Area = None\n    book_time: str = None  # hh:mm or dontcare\n    book_day: DayOfWeek = None\n    book_people: int = None\n\n\n@dataclass\nclass Taxi:\n    destination: str = None\n    leave_from: str = None\n    depart_time: str = None  # hh:mm or dontcare\n    arrive_by_time: str = None  # hh:mm or dontcare\n\n\n@dataclass\nclass BeliefState:\n    hotel: Hotel = None\n    train: Train = None\n    attraction: Attraction = None\n    restaurant: Restaurant = None\n    taxi: Taxi = None\n\n\nclass DialogueAgent(abc.ABC):\n\n    state: BeliefState\n\n    @abc.abstractmethod\n    def find_hotel(self, name: str = None, price_range: PriceRange = None, type: HotelType = None,\n                    parking: Option = None, book_number_of_days: int = None, book_day: DayOfWeek = None,\n                    book_people: int = None, area: Area = None, stars: Union[int, Literal[\"dontcare\"]] = None,\n                    internet: Option = None) -> Hotel:\n        pass\n\n    @abc.abstractmethod\n    def find_train(self, destination: str = None, leave_from: str = None, day: DayOfWeek = None,\n                   book_people: int = None, depart_time: str = None, arrive_by_time: str = None) -> Train:\n        pass\n\n    @abc.abstractmethod\n    def find_attraction(self, name: str = None, area: Area = None, type: AttractionType = None) -> Attraction:\n        pass\n\n    @abc.abstractmethod\n    def find_restaurant(self, name: str = None, food_type: str = None, price_range: PriceRange = None,\n                        area: Area = None, book_time: str = None, book_day: DayOfWeek = None,\n                        book_people: int = None, ) -> Restaurant:\n        pass\n\n    @abc.abstractmethod\n    def find_taxi(self, destination: str = None, leave_from: str = None, depart_time: str = None,\n                  arrive_by_time: str = None) -> Taxi:\n        pass\n\n    def get_state(self) -> BeliefState:\n        return state\n\n\nif __name__ == '__main__':\n    agent = DialogueAgent()\n    state = BeliefState()\n\n    \n\n    #### Example 1 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"area\": \"east\", \"internet\": \"yes\", \"type\": \"hotel\"}, \"attraction\": {\"type\": \"museum\", \"area\": \"centre\"}})\n    print(\"system: i have the express by holiday inn cambridge that matches your preferences . would you like me to book it for you ?\")\n    print(\"user: does the hotel include free parking ?\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(name=\"express by holiday inn cambridge\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 2 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"parking\": \"yes\", \"price_range\": \"expensive\", \"internet\": \"yes\", \"type\": \"hotel\"}})\n    print(\"system: i like the express by holiday inn cambridge . can i get you some more information on that 1 ?\")\n    print(\"user: no , let s just book it .\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(name=\"express by holiday inn cambridge\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 3 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"parking\": \"yes\", \"stars\": 2, \"type\": \"hotel\"}})\n    print(\"system: the ashley hotel , lovell lodge and express by holiday inn cambridge match your criteria . would you like more info on them ?\")\n    print(\"user: i need a guest house .\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(type=\"guest house\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 4 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"name\": state.hotel.name, \"area\": \"east\", \"internet\": \"yes\", \"type\": \"hotel\"}})\n    print(\"system: yes it does . would you like me to book it ?\")\n    print(\"user: no , thank you , that s all i need .\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.update({})  # no change\n"}, 
        {
            "role": "user", "content": "\n    #### Example 5 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"area\": \"east\", \"price_range\": \"expensive\"}, \"attraction\": {\"name\": state.attraction.name}})\n    print(\"system: i have 1 listing for express by holiday inn cambridge , is that okay for you ?\")\n    print(\"user: that works fine .\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(name=\"express by holiday inn cambridge\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 6 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"book_day\": \"saturday\", \"book_people\": 1, \"book_number_of_days\": 4, \"name\": state.hotel.name, \"area\": \"east\", \"stars\": 2, \"internet\": \"yes\", \"type\": \"hotel\"}, \"attraction\": {\"area\": \"south\"}, \"taxi\": {\"depart_time\": state.taxi.depart_time, \"leave_from\": state.taxi.leave_from}})\n    print(\"system: the south has cinema , entertainment , museum , nightclub , park and theatre . which do you prefer ?\")\n    print(\"user: can you make a recommendation ?\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.attraction = agent.find_attraction(type=\"dontcare\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 7 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"name\": state.hotel.name, \"area\": \"east\", \"internet\": \"yes\", \"type\": \"hotel\"}, \"restaurant\": {\"food_type\": state.restaurant.food_type, \"price_range\": \"moderate\", \"area\": \"east\"}})\n    print(\"system: okay the pizza hut fen ditton fits that criteria . it is located in cambridge retail park .\")\n    print(\"user: could you provide me with the address , phone number and postcode ?\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.restaurant = agent.find_restaurant(name=\"pizza hut fenditton\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 8 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"area\": \"east\", \"internet\": \"yes\", \"type\": \"hotel\"}})\n    print(\"system: i have found express by holiday inn cambridge . 1517 norman way , coldhams business park\")\n    print(\"user: does that hotel include free parking and have 2 stars ?\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(name=\"express by holiday inn cambridge\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 9 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"name\": state.hotel.name, \"area\": \"east\", \"price_range\": \"expensive\"}, \"attraction\": {\"name\": state.attraction.name}})\n    print(\"system: alright . they are at 01223866800 at the property is on 1517 norman way , coldhams business park . would you like a reservation ?\")\n    print(\"user: what s the star rating ? i want to stay at a 4 star place .\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(stars=\"4\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 10 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"area\": \"dontcare\", \"price_range\": \"expensive\", \"stars\": 2}})\n    print(\"system: okay , the express by holiday inn cambridge matches the criteria . the address is 1517 norman way , coldhams business park . would you like me to book a reservation ?\")\n    print(\"user: does it have free parking\")\n"}, 
        {
            "role": "assistant", "content": "    agent.state.hotel = agent.find_hotel(name=\"express by holiday inn cambridge\", parking=\"yes\")\n"}, 
        {
            "role": "user", "content": "\n    #### Example 11 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"name\": state.hotel.name}})\n    print(\"system: i have the express by holiday inn cambridge located on 1517 norman way , coldhams business park . their phone number is 01223866800 . would you like to know anything else ?\")\n    print(\"user: yes , could you book the hotel room for me for 7 people ?\")\n"},
        
    ]

In [3]:
# model = LLM(model="/data1/home/haesungpyun/models/Meta-Llama-3-70B-Instruct-AWQ", quantization='awq', enforce_eager=True)
 
model = LLM(model="../models/Meta-Llama-3-70B-Instruct-GPTQ", quantization='gptq', enforce_eager=False)

# model = LLM(model="/data1/home/haesungpyun/models/Meta-Llama-3-70B-Instruct-EXL2", enforce_eager=False)

INFO 08-25 18:29:52 gptq_marlin.py:91] Detected that the model can run with gptq_marlin, however you specified quantization=gptq explicitly, so forcing gptq. Use quantization=gptq_marlin for faster inference
WARNING 08-25 18:29:52 config.py:241] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 08-25 18:29:52 llm_engine.py:174] Initializing an LLM engine (v0.5.2) with config: model='../models/Meta-Llama-3-70B-Instruct-GPTQ', speculative_config=None, tokenizer='../models/Meta-Llama-3-70B-Instruct-GPTQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_c

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 08-25 18:30:48 model_runner.py:266] Loading model weights took 37.0657 GB
INFO 08-25 18:30:54 gpu_executor.py:86] # GPU blocks: 6445, # CPU blocks: 819
INFO 08-25 18:30:57 model_runner.py:1007] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-25 18:30:57 model_runner.py:1011] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-25 18:31:37 model_runner.py:1208] Graph capturing finished in 40 secs.


In [11]:
dataset[498]['prompt']

{'10-shot': [{'role': 'system',
   'content': "**Task:** You are an expert in Dialogue State Tracking (DST) focused on managing and updating the dialogue state change based on system-user interactions. The dialogue state represents the user's preferences and booking details across different domains: Hotel, Train, Attraction, Restaurant, and Taxi.\n\n"},
  {'role': 'user',
   'content': '**Guidelines:**\n\n    1. **Hotel:**\n        - **Slots:** name (string), pricerange (PriceRange), type (HotelType), parking (Option), book stay (integer), book day (DayOfWeek), book people (integer), area (Area), stars (integer between 0 and 5 or "dontcare"), internet (Option).\n        - **Valid Values:**\n            - PriceRange: "dontcare", "cheap", "moderate", "expensive".\n            - HotelType: "hotel", "guest house", "dontcare".\n            - Option: "yes", "no", "dontcare".\n            - DayOfWeek: "monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday".\n            -

: 

In [7]:
tokenizer = model.get_tokenizer()
stop_token_ids = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
stop_sequence = ['--',  ';', '#', '\n']
samplig_params = SamplingParams(n=1, best_of=1, max_tokens=120, temperature=0, stop=stop_sequence,stop_token_ids=stop_token_ids)

prompt_token_ids = tokenizer.apply_chat_template(
                    prompts_string,
                    add_generation_prompt=True,
                    return_tensors='pt').int()
prompts = tokenizer.batch_decode(prompt_token_ids)
# inputs = cast(Union[PromptStrictInputs, Sequence[PromptStrictInputs]],
#             prompts)

# start = datetime.now()
# outputs = model.generate(prompts, sampling_params=samplig_params)
# running_secs = (datetime.now() - start).seconds
# print(outputs[0].outputs[0].text)
# print(running_secs)

In [8]:
outputs = model.generate(prompts, sampling_params=samplig_params)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.60s/it, est. speed input: 1074.22 toks/s, output: 7.31 toks/s]


In [10]:
dataset[498]['slot_values']

{'hotel-internet': 'yes', 'hotel-type': 'hotel'}

In [9]:
outputs[0].outputs[0].text

"{'attraction-name': 'adc theatre', 'attraction-type': 'theatre'}"

In [9]:
prompts_10 = prompts*10
print(len(prompts_10))
print()

start = datetime.now()
for _ in range(3):
    for _ in range(3):
        outputs = model.generate(prompts_10, sampling_params=samplig_params)
running_secs = (datetime.now() - start).seconds
print(running_secs)

10



Processed prompts:   0%|          | 0/10 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 10/10 [00:24<00:00,  2.47s/it, est. speed input: 929.49 toks/s, output: 9.70 toks/s]

192


In [ ]:
prompts_30 = prompts * 30
print(len(prompts_30))
print()

start = datetime.now()
for _ in range(3):
    outputs = model.generate(prompts_30, sampling_params=samplig_params)
running_secs = (datetime.now() - start).seconds
print(running_secs)

In [ ]:
prompts_30_3 = prompts_30 * 3
print(len(prompts_30_3))
print()
start = datetime.now()
outputs = model.generate(prompts_30_3, sampling_params=samplig_params)
running_secs = (datetime.now() - start).seconds
print(running_secs)

Processed prompts: 100%|██████████| 90/90 [03:13<00:00,  2.15s/it, est. speed input: 1067.71 toks/s, output: 11.15 toks/s]

194


In [1]:
import sys, os
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from exllamav2 import ExLlamaV2, ExLlamaV2Config, ExLlamaV2Cache, ExLlamaV2Tokenizer, Timer
from exllamav2.generator import ExLlamaV2DynamicGenerator

model_dir = "/data1/home/haesungpyun/models/Meta-Llama-3-70B-Instruct-EXL2"
config = ExLlamaV2Config(model_dir)
config.arch_compat_overrides()
model = ExLlamaV2(config)
cache = ExLlamaV2Cache(model, max_seq_len = 32768, lazy = True)
model.load_autosplit(cache, progress = True)

print("Loading tokenizer...")
tokenizer = ExLlamaV2Tokenizer(config)

# Initialize the generator with all default parameters

generator = ExLlamaV2DynamicGenerator(
    model = model,
    cache = cache,
    tokenizer = tokenizer,
)

/home/haesungpyun/anaconda3/envs/torch2.1_clone/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Output()

Loading tokenizer...


In [2]:
from exllamav2.generator import ExLlamaV2Sampler

settings = ExLlamaV2Sampler.Settings(
    temperature = 0,
    top_k = 1,
    top_p = 0,
    top_a = 0,
    typical = 0,
    skew = 0,
    token_repetition_penalty = 1.0,
    token_frequency_penalty = 0.0,
    token_presence_penalty = 0.0,
    smoothing_factor = 0.0,
)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)
prompts = tokenizer.batch_decode(tokenizer.apply_chat_template(prompts_string, add_generation_prompt=True, return_tensors='pt'))
prompts

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are an expert in Dialogue State Tracking(DST) and python coding. And also you are a talented prompt engineer. \n Please generate the shots which are the most<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nimport abc\nfrom dataclasses import dataclass\nfrom typing import Literal, Union\n\nPriceRange = Literal["dontcare", "cheap", "moderate", "expensive"]\nHotelType = Literal["hotel", "guest house", "dontcare"]\nOption = Literal["yes", "no", "dontcare"]\nDayOfWeek = Literal["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]\nArea = Literal["dontcare", "centre", "east", "north", "south", "west"]\n\n\n@dataclass\nclass Hotel:\n    name: str = None\n    price_range: PriceRange = None\n    type: HotelType = None\n    parking: Option = None\n    book_number_of_days: int = None\n    book_day: DayOfWeek = None\n    book_people: int = None\n    area: Area = None\n    stars: Union[int, Literal["d

In [ ]:
stop_token_ids = [tokenizer.eos_token_id, tokenizer.encode("<|eot_id|>").item()]
stop_sequence = ['--',  ';', '#', '\n']
# samplig_params = SamplingParams(n=1, best_of=1, max_tokens=120, temperature=0, stop=stop_sequence,stop_token_ids=stop_token_ids)

# prompt_token_ids = tokenizer.apply_chat_template(
#                     prompts_string,
#                     add_generation_prompt=True,
#                     return_tensors='pt').int()
# prompts = tokenizer.batch_decode(prompt_token_ids)

In [10]:
from datetime import datetime

max_new_tokens = 120

# Warmup generator. The function runs a small completion job to allow all the kernels to fully initialize and
# autotune before we do any timing measurements. It can be a little slow for larger models and is not needed
# to produce correct output.

generator.warmup()

# Generate one completion, using default settings

with Timer() as t_single:
    start = datetime.now()
    output = generator.generate(prompt = prompts[0], max_new_tokens = max_new_tokens, add_bos = True)
    running_secs = (datetime.now() - start).seconds

print("-----------------------------------------------------------------------------------")
print("- Single completion")
print("Running Seconds : ", running_secs)
print("-----------------------------------------------------------------------------------")
print(output)
print()


-----------------------------------------------------------------------------------
- Single completion
Running Seconds :  6
-----------------------------------------------------------------------------------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in Dialogue State Tracking(DST) and python coding. And also you are a talented prompt engineer. 
 Please generate the shots which are the most<|eot_id|><|start_header_id|>user<|end_header_id|>

import abc
from dataclasses import dataclass
from typing import Literal, Union

PriceRange = Literal["dontcare", "cheap", "moderate", "expensive"]
HotelType = Literal["hotel", "guest house", "dontcare"]
Option = Literal["yes", "no", "dontcare"]
DayOfWeek = Literal["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
Area = Literal["dontcare", "centre", "east", "north", "south", "west"]


@dataclass
class Hotel:
    name: str = None
    price_range: PriceRange = None
    type: HotelType = 

In [14]:
prompts_10 = prompts*10
prompts_10
prompts_30 = prompts*30*3

In [15]:
with Timer() as t_batched:
    start = datetime.now()
    outputs = generator.generate(prompt = prompts_30, max_new_tokens = max_new_tokens, add_bos = True)
    running_secs = (datetime.now() - start).seconds

print('-----------------------------------------------------------------------------------')
print("Running Seconds: ", running_secs)
print('-----------------------------------------------------------------------------------')

for idx, output in enumerate(outputs):
    print("-----------------------------------------------------------------------------------")
    print(f"- Batched completion #{idx + 1}")
    print("-----------------------------------------------------------------------------------")
    print(output)
    print()

print("-----------------------------------------------------------------------------------")
print(f"speed, bsz 1: {max_new_tokens / t_single.interval:.2f} tokens/second")
print(f"speed, bsz {len(prompts)}: {max_new_tokens * len(prompts) / t_batched.interval:.2f} tokens/second")

-----------------------------------------------------------------------------------
Running Seconds:  118
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
- Batched completion #1
-----------------------------------------------------------------------------------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in Dialogue State Tracking(DST) and python coding. And also you are a talented prompt engineer. 
 Please generate the shots which are the most<|eot_id|><|start_header_id|>user<|end_header_id|>

import abc
from dataclasses import dataclass
from typing import Literal, Union

PriceRange = Literal["dontcare", "cheap", "moderate", "expensive"]
HotelType = Literal["hotel", "guest house", "dontcare"]
Option = Literal["yes", "no", "dontcare"]
DayOfWeek = Literal["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
Ar

In [16]:
running_secs

118

: 

In [1]:
import torch
from transformers import AutoTokenizer, LlamaForCausalLM

In [2]:
model = LlamaForCausalLM.from_pretrained(        
    'meta-llama/Meta-Llama-3-70B-Instruct',
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=False
)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3-8B-Instruct')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
inputs = tokenizer.apply_chat_template(
    [{
        'role': 'user',
        'content':'hello'
    }],
    add_generation_prompt=True,
    return_tensors="pt"
)

In [13]:
tokenizer.batch_decode(model.generate(
    inputs
))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/haesungpyun/anaconda3/envs/refpydst/lib/python3.11/site-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


["<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nhello<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHello! It's nice to meet you. Is there something I can help you with, or would you like to chat?<|eot_id|>"]

In [3]:
engine='meta-llama/Meta-Llama-3-70B-Instruct'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = LlamaForCausalLM.from_pretrained(
    engine,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=False
)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
messages_chat

tensor([[128000, 128006,   9125,  ...,  78191, 128007,    271]])

In [8]:
tokenizer = AutoTokenizer.from_pretrained(engine)
tokenizer.pad_token_id = tokenizer.eos_token_id

stop_token_ids = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
stop_sequence = ['--',  ';', '#', '\n']

input_ids = tokenizer.apply_chat_template(
                    prompts_string,
                    add_generation_prompt=True,
                    return_tensors="pt")

sequences, logits = [], []

start = datetime.now()
result = model.generate(
    input_ids,
    max_new_tokens=120,
    eos_token_id=stop_token_ids,
    do_sample=True,
    top_p=0.9,
    output_scores= True,
    return_dict_in_generate=True,
    stop_strings=stop_sequence,
    tokenizer=tokenizer,
    output_logits=True,
    )
running_secs = (datetime.now() - start).seconds

sequences.append(result['sequences'][0])
logits.append(result['logits'])
print(sequences)
print(running_secs)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/haesungpyun/anaconda3/envs/torch2.1_clone/lib/python3.10/site-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to 

[tensor([128000, 128006,   9125,  ...,     22,      8, 128009])]
11


In [11]:
len(sequences[0])/11

212.54545454545453

In [ ]:
engine='meta-llama/Meta-Llama-3-70B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(engine)
tokenizer.pad_token_id = tokenizer.eos_token_id
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = LlamaForCausalLM.from_pretrained(
    engine,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=False
)
# Define the directory where the model will be saved
save_directory = "/data1/home/haesungpyun/_8b_quantized"

# Save the model and tokenizer to the specified directory
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

In [ ]:
llm = LLM(model='hacastle12/Meta-Llama-3-70B-Instruct-BitsAndBytes')

In [ ]:
engine_args = EngineArgs(
    model='hacastle12/Meta-Llama-3-70B-Instruct-BitsAndBytes',
    # quantization='bitsandbytes',
    # qlora_adapter_name_or_path='timdettmers/qlora-flan-7b',
    # load_format='bitsandbytes',
    # enable_lora=False,
    # max_lora_rank=64,
    # set it only in GPUs of limited memory
    enforce_eager=True)
llm_engine = LLMEngine.from_engine_args(
            engine_args, usage_context=UsageContext.LLM_CLASS)
request_counter = Counter()


In [ ]:

inputs = cast(
        Union[PromptStrictInputs, Sequence[PromptStrictInputs]],
        messages)

In [ ]:
for idx, msg in enumerate(messages):
        llm_engine.add_request(
                request_id=str(idx),
                inputs={"prompt_token_ids":msg},
                params=samplig_params,
                lora_request=None
        )
outputs = []
while llm_engine.has_unfinished_requests():
    step_outputs = llm_engine.step()
    for output in step_outputs:
        if output.finished:
            outputs.append(output)
outputs = sorted(outputs, key=lambda x: int(x.request_id))

In [ ]:
outputs[0].outputs[0].text

In [ ]:
test_config= {
"name": "qlora_inference_example",
'model': "meta-llama/Meta-Llama-3-8B-Instruct",
'quantization': "bitsandbytes",
'lora_repo': 'timdettmers/qlora-flan-7b'
}

engine_args = EngineArgs(
    model='/data1/home/haesungpyun/llama-3-70b-instruct-awq',
    quantization='AWQ',
    # enable_lora=True,
    # max_lora_rank=64,
    # set it only in GPUs of limited memory
    enforce_eager=True)
engine = LLMEngine.from_engine_args(engine_args)


# engine_args = EngineArgs(model='mcasperhansen/llama-3-8b-instruct-awq', quantization='AWQ', enforce_eager=True)

In [ ]:
tokenizer = engine.get_tokenizer()

In [ ]:
messages.__len__()

In [ ]:
test_prompts = [(messages[0], samplig_params, None)]
request_id = 0
while test_prompts or engine.has_unfinished_requests():
        if test_prompts:
            prompt, sampling_params, lora_request = test_prompts.pop(0)
            engine.add_request(str(request_id),
                               prompt,
                               sampling_params,
                               lora_request=lora_request)
            request_id += 1

        request_outputs = engine.step()
        for request_output in request_outputs:
            if request_output.finished:
                print("----------------------------------------------------")
                print(f"Prompt: {request_output.prompt}")
                print(f"Output: {request_output.outputs[0].text}")

In [ ]:
engine.add_request(str(0), messages[0], samplig_params, lora_request=None)

In [ ]:
engine.has_unfinished_requests()

In [ ]:
while engine.has_unfinished_requests():
    output = engine.step()
    for o in output:
        print(o.finished)

In [ ]:
output[0].outputs[0].text

In [ ]:
"""
This example shows how to use LoRA with different quantization techniques
for offline inference.

Requires HuggingFace credentials for access.
"""

import gc
from typing import List, Optional, Tuple

import torch
from huggingface_hub import snapshot_download

from vllm import EngineArgs, LLMEngine, RequestOutput, SamplingParams
from vllm.lora.request import LoRARequest


"""Main function that sets up and runs the prompt processing."""
test_config= {
"name": "qlora_inference_example",
'model': "meta-llama/Meta-Llama-3-8B-Instruct",
'quantization': "bitsandbytes",
'lora_repo': 'timdettmers/qlora-flan-7b'
}

engine_args = EngineArgs(
    model=test_config['model'],
    quantization=test_config['quantization'],
    qlora_adapter_name_or_path=test_config['lora_repo'],
    load_format=test_config['quantization'],
    enable_lora=True,
    max_lora_rank=64,
    # set it only in GPUs of limited memory
    enforce_eager=True)
engine = LLMEngine.from_engine_args(engine_args)

lora_path = snapshot_download(repo_id=test_config['lora_repo'])


In [ ]:
test_prompts = [
     ('Say Hello to me',SamplingParams(temperature=0.0,
                logprobs=1,
                prompt_logprobs=1,
                max_tokens=128), None),
    # ("my name is", SamplingParams(temperature=0.0,
    #             logprobs=1,
    #             prompt_logprobs=1,
    #             max_tokens=128),
    # LoRARequest("lora-test-1", 1, lora_path))
    ]
request_id = 0
while test_prompts or engine.has_unfinished_requests():
        if test_prompts:
            prompt, sampling_params, lora_request = test_prompts.pop(0)
            engine.add_request(str(request_id),
                               prompt,
                               sampling_params,
                               lora_request=lora_request)
            request_id += 1

        request_outputs: List[RequestOutput] = engine.step()
        for request_output in request_outputs:
            if request_output.finished:
                print("----------------------------------------------------")
                print(f"Prompt: {request_output.prompt}")
                print(f"Output: {request_output.outputs[0].text}")

In [ ]:
request_outputs[-1].outputs

In [ ]:
model_id = "meta-llama/Meta-Llama-3-70B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
messages = [
            {
                "role": "system",
                "content": "You are an expert in Dialogue State Tracking(DST) and python coding. And also you are a talented prompt engineer. \n Please generate the shots which are the most "
            },
            {
                "role": "user",
                "content": "import abc\nfrom dataclasses import dataclass\nfrom typing import Literal, Union\n\nPriceRange = Literal[\"dontcare\", \"cheap\", \"moderate\", \"expensive\"]\nHotelType = Literal[\"hotel\", \"guest house\", \"dontcare\"]\nOption = Literal[\"yes\", \"no\", \"dontcare\"]\nDayOfWeek = Literal[\"monday\", \"tuesday\", \"wednesday\", \"thursday\", \"friday\", \"saturday\", \"sunday\"]\nArea = Literal[\"dontcare\", \"centre\", \"east\", \"north\", \"south\", \"west\"]\n\n\n@dataclass\nclass Hotel:\n    name: str = None\n    price_range: PriceRange = None\n    type: HotelType = None\n    parking: Option = None\n    book_number_of_days: int = None\n    book_day: DayOfWeek = None\n    book_people: int = None\n    area: Area = None\n    stars: Union[int, Literal[\"dontcare\"]] = None  # between 0 and 5 or dontcare\n    internet: Option = None\n\n\n@dataclass\nclass Train:\n    destination: str = None\n    leave_from: str = None\n    day: DayOfWeek = None\n    book_people: int = None\n    depart_time: str = None  # hh:mm or dontcare\n    arrive_by_time: str = None  # hh:mm or dontcare\n\n\nAttractionType = Literal[\"architecture\", \"boat\", \"church\", \"cinema\", \"college\", \"concert hall\", \"entertainment\",\n                         \"hotspot\", \"multiple sports\", \"museum\", \"nightclub\", \"park\", \"special\", \"swimming pool\",\n                         \"theatre\", \"dontcare\"]\n\n\n@dataclass\nclass Attraction:\n    name: str = None\n    area: Area = None\n    type: AttractionType = None\n\n\n@dataclass\nclass Restaurant:\n    name: str = None\n    food_type: str = None\n    price_range: PriceRange = None\n    area: Area = None\n    book_time: str = None  # hh:mm or dontcare\n    book_day: DayOfWeek = None\n    book_people: int = None\n\n\n@dataclass\nclass Taxi:\n    destination: str = None\n    leave_from: str = None\n    depart_time: str = None  # hh:mm or dontcare\n    arrive_by_time: str = None  # hh:mm or dontcare\n\n\n@dataclass\nclass BeliefState:\n    hotel: Hotel = None\n    train: Train = None\n    attraction: Attraction = None\n    restaurant: Restaurant = None\n    taxi: Taxi = None\n\n\nclass DialogueAgent(abc.ABC):\n\n    state: BeliefState\n\n    @abc.abstractmethod\n    def find_hotel(self, name: str = None, price_range: PriceRange = None, type: HotelType = None,\n                    parking: Option = None, book_number_of_days: int = None, book_day: DayOfWeek = None,\n                    book_people: int = None, area: Area = None, stars: Union[int, Literal[\"dontcare\"]] = None,\n                    internet: Option = None) -> Hotel:\n        pass\n\n    @abc.abstractmethod\n    def find_train(self, destination: str = None, leave_from: str = None, day: DayOfWeek = None,\n                   book_people: int = None, depart_time: str = None, arrive_by_time: str = None) -> Train:\n        pass\n\n    @abc.abstractmethod\n    def find_attraction(self, name: str = None, area: Area = None, type: AttractionType = None) -> Attraction:\n        pass\n\n    @abc.abstractmethod\n    def find_restaurant(self, name: str = None, food_type: str = None, price_range: PriceRange = None,\n                        area: Area = None, book_time: str = None, book_day: DayOfWeek = None,\n                        book_people: int = None, ) -> Restaurant:\n        pass\n\n    @abc.abstractmethod\n    def find_taxi(self, destination: str = None, leave_from: str = None, depart_time: str = None,\n                  arrive_by_time: str = None) -> Taxi:\n        pass\n\n    def get_state(self) -> BeliefState:\n        return state\n\n\nif __name__ == '__main__':\n    agent = DialogueAgent()\n    state = BeliefState()\n\n    \n\n    #### Example 1 ####\n    agent.state = BeliefState.from_dict({\"hotel\": {\"area\": \"north\", \"parking\": \"yes\", \"price_range\": \"moderate\", \"stars\": 4, \"internet\": \"yes\"}})\n    print(\"system: there are 6 hotel -s in the north part of the city that fall in the moderate price range , have free wifi and parking , and have 4 stars . any other specification -s ?\")\n    print(\"user: i am not looking to book yet , can you tell me their price range -s ?\")\n"
            },
            {
                "role": "assistant",
                "content": "    agent.state.update({})  # no change\n"
            },
            {
                "role": "user",
                "content": "\n    #### Example 11 ####\n    agent.state = BeliefState.from_dict({})\n    print(\"user: i looking for information about a hotel in the moderate price range that include -s free wifi .\")\n"
            }
        ]


In [ ]:
stop_sequences = ['--', '\n', ';', '#']
terminators =  [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")    
]
input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

sequences, logits = [], []
for _ in range(2):
    # result = openai.completions.create(**args)
    result = model.generate(
        input_ids,
        max_new_tokens=120,
        eos_token_id=terminators,
        do_sample=True,
        top_p=0.9,
        output_scores= True,
        return_dict_in_generate=True,
        stop_strings=stop_sequences,
        tokenizer=tokenizer,
        output_logits=True,
        )

    sequences.append(result['sequences'][0])
    logits.append(result['logits'])


In [ ]:
[len(logit) for logit in logits]

In [ ]:
[torch.stack(logit).softmax(dim=-1).max(dim=-1)[0].log().sum().item() for logit in logits]

In [ ]:
dict(zip(
    [tokenizer.decode(seq[input_len:], skip_special_tokens=True) for seq in sequences],
    [torch.stack(logit).softmax(dim=-1).max(dim=-1)[0].log().sum(0).item() for logit in logits]))

In [ ]:
result['sequences'].shape

In [ ]:
input_len = input_ids.shape[-1]

dict(
    zip(
        [tokenizer.decode(seq[input_len:], skip_special_tokens=True) for seq in result['sequences']] ,
        torch.stack(result['scores']).softmax(dim=-1).max(dim=-1)[0].log().sum(0)
        )
    )

In [ ]:
torch.stack(result['scores']).softmax(dim=-1).max(dim=-1)[0].log().sum(0)

In [ ]:
# list of torch.tensor to a tensor
from torch import nn
torch.max(nn.functional.softmax(torch.stack(result['logits']), dim=-1), dim=-1)[0].log().sum().item()

In [ ]:
torch.stack(result['logits']).softmax(dim=-1).max(dim=-1)[0].log().sum().item()

In [ ]:
from transformers import StoppingCriteria

stop_sequences =  [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")    
]
input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

result_2 = model.generate(
    input_ids,
    max_new_tokens=120,
    eos_token_id=stop_sequences,
    stop_strings=["\n\n", "#", "print("],
    do_sample=False,
    temperature=0.0,
    output_scores= True,
    return_dict_in_generate=True,
    tokenizer=tokenizer
)
tokenizer.decode(result_2['sequences'][0][input_ids.shape[-1]:], skip_special_tokens=True)

In [ ]:
result_2['sequences'][0][input_ids.shape[-1]:].__len__()

In [ ]:
result_2['sequences'][0][input_ids.shape[-1]:]

In [ ]:
from torch import nn
a = 1
a *= max(nn.functional.softmax(logits) for logits in result_2['scores'])

In [ ]:
a = 1
for logits in result_2['scores']:
    a*=torch.max(nn.functional.softmax(logits, dim=-1))

In [ ]:
a

In [ ]:
result_2['sequences'], outputs[0][input_ids.shape[-1]:], response

In [ ]:
tokenizer.decode(result_2['sequences'][0][input_ids.shape[-1]:], skip_special_tokens=False), 

In [ ]:
result_2['logits']